# 20241201
---

## 파이프라인

In [14]:
import pandas as pd

train = pd.read_csv('./datasets/customer_train.csv')
test = pd.read_csv('./datasets/customer_test.csv')

print(train.info())

""" 출력 결과
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   회원ID     3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   방문일수     3500 non-null   int64  
 7   방문당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
 10  성별       3500 non-null   int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 300.9+ KB
"""

print(test.info())

""" 출력 결과
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2482 entries, 0 to 2481
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   회원ID     2482 non-null   int64  
 1   총구매액     2482 non-null   int64  
 2   최대구매액    2482 non-null   int64  
 3   환불금액     871 non-null    float64
 4   주구매상품    2482 non-null   object 
 5   주구매지점    2482 non-null   object 
 6   방문일수     2482 non-null   int64  
 7   방문당구매건수  2482 non-null   float64
 8   주말방문비율   2482 non-null   float64
 9   구매주기     2482 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 194.0+ KB
"""

X_train = train.copy()
X_test = test.copy()

# (1) 데이터 전처리
## 결측치 처리
target_column = X_train['환불금액']
target_column.fillna(0, inplace=True)    # 0, 최빈값, 평균 등으로 채우기

## 데이터 스케일링
### -> RF, LightBGM, XGBoost는 결정 트리 기반 알고리즘이기 때문에 필요 X

## 이상치 처리
"""
target_column = X_train['변수명']

Q1, Q3 = target_column.quantile([0.25, 0.75])
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

cond = (target_column < lower) | (target_column > upper)   # 이상치 조건
target_column = target_column.median()   # 중앙값으로 대체
"""

## 불필요한 변수 제거
X_train = X_train.drop('회원ID', axis=1)
X_test = X_test.drop('회원ID', axis=1)

# (2) 독립/종속 변수 분리
y = train['성별']
X = train.drop('성별', axis=1)

# (3) 독립/종속 변수 인코딩
X_encoded = pd.get_dummies(X)   # ⭐💡 반드시 독립/종속 변수 분리 후 진행한다! 
X_test_encoded = pd.get_dummies(X_test)

print(set(X_test['주구매상품']) - set(X_train['주구매상품']))
print(set(X_test['주구매지점']) - set(X_train['주구매지점']))

X_test_encoded = X_test_encoded.reindex(columns=X_encoded.columns, fill_value=0)   # 훈련, 테스트 데이터 열 구성 동일하게 맞추기

# (4) 데이터 분할
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_encoded, y, test_size=0.2)
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

""" 출력 결과
(2800, 74) (700, 74) (2800,) (700,)
"""

# (5) 모델링
## [1] RandomForest
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

model_rf = RandomForestClassifier()   # RandomForestRegressor()
model_rf.fit(X_tr, y_tr)
# model = RandomForestRegressor()

## [2] XGBoost
from xgboost import XGBClassifier, XGBRegressor

model_xgb = XGBClassifier(eval_metric='logloss')   # XGBRegressor(eval_metric='rmse')
model_xgb.fit(X_tr, y_tr)

## [3] LightGBM
from lightgbm import LGBMClassifier, LGBMRegressor

model_lgbm = LGBMClassifier()    # LGBMRegressor()
model_lgbm.fit(X_tr, y_tr)

# (6) 예측
pred_rf = model_rf.predict(X_val)
pred_xgb = model_xgb.predict(X_val)
pred_lgbm = model_lgbm.predict(X_val)

# (7) 평가
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.metrics import root_mean_squared_error, mean_squared_error

score_rf = f1_score(y_val, pred_rf)
score_xgb = roc_auc_score(y_val, pred_xgb)
score_lgbm = roc_auc_score(y_val, pred_lgbm)

print(score_rf)
print(score_xgb)
print(score_lgbm)

""" 출력 결과
0.6054263565891473
0.5919896640826873
0.5943583118001723
"""

# (8) 테스트 데이터 예측
pred_rf = model_rf.predict(X_test_encoded)
pred_xgb = model_xgb.predict(X_test_encoded)
pred_lgbm = model_lgbm.predict(X_test_encoded)

print(pred_rf)
print(pred_xgb)
print(pred_lgbm)

""" 출력 결과
[1 0 0 ... 1 0 1]
[0 0 0 ... 0 0 1]
[0 0 0 ... 1 1 0]
"""

# (9) CSV 내보내기
result = pd.DataFrame({
    'pred_rf': pred_rf,
})
print(result)

""" 출력 결과
      pred
0        0
1        0
2        0
3        0
4        0
...    ...
2477     0
2478     0
2479     1
2480     1
2481     0

[2482 rows x 1 columns]
"""

result.to_csv('./outputs/modeling_pipeline.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   회원ID     3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   방문일수     3500 non-null   int64  
 7   방문당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
 10  성별       3500 non-null   int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 300.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2482 entries, 0 to 2481
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   회원ID     2482 non-null   int64  
 1   총구매액     2482 non-null   int64  
 2   최대구매액    2482 non-null   int64  
 3   환불금액     871 non-null    flo

In [4]:
from scipy.stats import pearsonr

df = pd.DataFrame({
    'X': [10, 20, 30, 40, 50],
    'Y': [15, 25, 35, 45, 55]
})

corr = df['X'].corr(df['Y'])    # 피어슨 상관계수 값 확인
print(f"피어슨 상관 계수: {corr}")


""" 출력 결과
피어슨 상관 계수: 1.0
"""

피어슨 상관 계수: 1.0


' 출력 결과\n피어슨 상관 계수: 1.0\n'

In [ ]:
# 피어슨 상관 계수(Pearson Correlation Coefficient) 확인하기
## 방법 1 : 피어슨 상관 계수만 확인
import pandas as pd

df = pd.DataFrame({
    'X': [1, 2, 3, 4, 5],
    'Y': [5, 4, 3, 2, 1]
})

corr = df['X'].corr(df['Y'])    # 피어슨 상관계수 확인
print(corr)

""" 출력 예시
-0.9999999999999999
"""

## 방법 2 : 피어슨 상관 계수, p-value 확인
from scipy.stats import pearsonr

x = [1, 2, 3, 4, 5]
y = [5, 4, 3, 2, 1]

corr, p_value = pearsonr(x, y)   # 피어슨 상관계수, pvalue 확인

print(f"피어슨 상관 계수: {corr}")
print(f"p-값: {p_value}")

""" 출력 예시
피어슨 상관 계수: -1.0
p-값: 0.0
"""

-0.9999999999999999
피어슨 상관 계수: -1.0
p-값: 0.0


' 출력 예시\n270.45483621802725\n'